In [1]:
import sys
sys.path.append('../utils')

import tensorflow as tf
from stardist import gputools_available
from glob import glob

from functions import *

In [2]:
pth_HE_tiles = r'path_to_H&E_tiles'

pth_mask_tiles = r'path_to_masks'

In [3]:
HE_tiles = read_tiles(pth_HE_tiles)
masks = read_masks(pth_mask_tiles)

In [4]:
ratio_validation_tiles = 0.4  # ratio of tiles that will be set aside for validation

tiles_train, masks_train, tiles_val, masks_val = split_train_val_set(HE_tiles, masks, ratio_validation_tiles)

In [5]:
# add flips and rotations to the images/masks pairs
HE_train_aug, masks_train_aug = augment_tiles(tiles_train, masks_train)

In [ ]:
print(f'{len(HE_train_aug)}')
print(f'{len(masks_train_aug)}')
print(f'{len(tiles_val)}')
print(f'{(len(masks_val)}')

In [7]:
# normalize H&E images by dividing by 255
HE_train_aug= normalize_images(HE_train_aug)
tiles_val = normalize_images(tiles_val)

In [ ]:
# look at training data
i = 50
show_HE_and_segmented(HE_train_aug[i], masks_train_aug[i])

In [ ]:
# look at validation
i = 4
show_HE_and_segmented(tiles_val[i], masks_val[i])

In [ ]:
# config GPU
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")
    
print(tf.__version__)

#Define the config by setting some parameter values
# 32 is a good default choice (see 1_data.ipynb)
n_rays = 32  #Number of radial directions for the star-convex polygon.

# Use OpenCL-based computations for data generator during training (requires 'gputools')
use_gpu = True and gputools_available()

# Predict on subsampled grid for increased efficiency and larger field of view
grid = (2,2)

n_channel = 1 if tiles_train[0].ndim == 2 else tiles_train[0].shape[-1]  #If no third dim. then number of channels = 1. Otherwise get the num channels from the last dim.


conf = Config2D (
    n_rays       = n_rays,
    grid         = grid,
    use_gpu      = use_gpu,
    n_channel_in = n_channel,
)

In [ ]:
# define hyperparameters, name model
lr = 0.001
epochs = 50
patience = 40

nm='Fallopian_monkey_test_tiles'
dt='_2_20_24'
outnm = nm + dt + '_lr_' + str(lr) + '_epochs_' + str(epochs) + '_pt_' + str(patience)
print(outnm)

In [12]:
# where do you want to save the model
outpth = r'path_to_save_model_to'

In [ ]:
# configure model
model = load_published_he_model(outpth, outnm)
model.config.train_learning_rate = lr
model.config.train_patch_size = (256,256)
model.config.train_reduce_lr={'factor': 0.5, 'patience': patience, 'min_delta': 0}
model.train(HE_train_aug, masks_train_aug, validation_data=(tiles_val, masks_val), epochs=epochs, steps_per_epoch=100)
model.optimize_thresholds(tiles_val, masks_val)

pth_log_train = fr"{outpth}\{outnm}\logs\train"
if not os.path.exists(pth_log_train): os.mkdir(pth_log_train)

pth_log = glob(os.path.join(pth_log_train,'*.v2'))[0]
loss = get_loss_data(str(pth_log), pth_log_train)